<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°08

**Objetivo**: Aplicar técnicas de **machine learning no supervisado** para explorar, procesar y analizar conjuntos de datos con variables numéricas y categóricas.

> **Nota**: Puede ayudarse de algún asistente virtual como **ChatGPT, Gemini** u otros, así como del autocompletado de **Google Colab**, para avanzar en este laboratorio debido a su extensión.


## Clustering


<img src="https://www.svgrepo.com/show/253022/car.svg" width = "300" align="center"/>



El conjunto de datos **`vehiculos_procesado_con_grupos.csv`** recopila información sobre diversas características relevantes de distintos vehículos. El propósito de este ejercicio es **clasificar los vehículos en diferentes categorías**, utilizando como base las variables descritas en la tabla de atributos.

El análisis presenta un desafío adicional debido a la **naturaleza mixta de los datos**: se incluyen tanto variables **numéricas** (ej. dimensiones, consumo, emisiones) como **categóricas** (ej. tipo de tracción, tipo de combustible), lo que requiere aplicar técnicas de preprocesamiento adecuadas antes de entrenar los modelos.

Como primer paso, procederemos a **cargar y explorar el conjunto de datos**, con el fin de familiarizarnos con su estructura y las características que servirán como base para la posterior clasificación.




**Descripción de los Datos:**

| **Nombre de la Columna**   | **Descripción**                                                                                                                                   |
|----------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| **year**                   | El año en que el vehículo fue fabricado.                                                                                                          |
| **desplazamiento**          | La capacidad volumétrica del motor en litros. Indica la cantidad de aire y combustible que puede desplazar el motor durante una revolución.       |
| **cilindros**               | El número de cilindros que tiene el motor. Los cilindros son las cámaras donde ocurre la combustión interna en los motores de los vehículos.       |
| **co2**                     | Emisiones de dióxido de carbono del vehículo, medido en gramos por kilómetro. Es una medida de las emisiones de gases de efecto invernadero.       |
| **clase_tipo**              | La clase o tipo de vehículo, como vehículos especiales, deportivos, etc.                                                                         |
| **traccion_tipo**           | Tipo de tracción del vehículo, ya sea tracción en dos ruedas, en cuatro ruedas o en todas las ruedas.                                             |
| **transmision_tipo**        | Tipo de transmisión del vehículo, como automática, manual, entre otros.                                                                          |
| **combustible_tipo**        | Tipo de combustible que utiliza el vehículo, como gasolina, diésel, eléctrico, híbrido, etc.                                                     |
| **tamano_motor_tipo**       | Clasificación del tamaño del motor (por ejemplo, pequeño, mediano o grande), que generalmente se basa en la capacidad de desplazamiento.           |
| **consumo_tipo**            | Clasificación del nivel de consumo de combustible del vehículo, indicando si es alto, bajo, o muy alto.                                           |
| **co2_tipo**                | Clasificación de las emisiones de CO2 del vehículo, indicando si es alto, bajo, o muy alto.                                                       |
| **consumo_litros_milla**    | El consumo de combustible del vehículo, medido en litros por milla. Indica la eficiencia del vehículo en términos de consumo de combustible.        |



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans


%matplotlib inline

sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/vehiculos_procesado_con_grupos.csv", sep=",")\
       .drop(
            ["fabricante",
             "modelo",
             "transmision",
             "traccion",
             "clase",
             "combustible",
             "consumo"],

          axis=1)

df.head()

,year,desplazamiento,cilindros,co2,clase_tipo,traccion_tipo,transmision_tipo,combustible_tipo,tamano_motor_tipo,consumo_tipo,co2_tipo,consumo_litros_milla
0,1984,2.5,4.0,522.764706,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.222671
1,1984,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
2,1985,2.5,4.0,555.437500,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.236588
3,1985,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
4,1987,3.8,6.0,555.437500,Coches Medianos,dos,Automatica,Premium,grande,alto,alto,0.236588


En este caso, no solo se tienen datos numéricos, sino que también categóricos. Además, tenemos problemas de datos **vacíos (Nan)**. Así que para resolver este problema, seguiremos varios pasos:

### 1.- Normalizar datos

- Cree un conjunto de datos con las variables numéricas, además, para cada dato vacía, rellene con el promedio asociado a esa columna. Finalmente, normalize los datos mediante el procesamiento **MinMaxScaler** de **sklearn**.
- Cree un conjunto de datos con las variables categóricas , además, transforme de variables categoricas a numericas ocupando el comando **get_dummies** de pandas ([referencia](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Explique a grande rasgo como se realiza la codificación de variables numéricas a categóricas.

- Junte ambos dataset en uno, llamado **df_procesado**.

In [ ]:
# FIXME

# Asumimos que el DataFrame original es df
# 1) NUMÉRICAS → imputación con media y MinMaxScaler a [0,1]
num_cols = df.select_dtypes(include=[np.number]).columns
df_num = df[num_cols].copy()
df_num = df_num.fillna(df_num.mean(numeric_only=True))          # imputar NaN con media de cada columna
minmax = MinMaxScaler()
df_num.loc[:, :] = minmax.fit_transform(df_num.values)          # normalizar

# 2) CATEGÓRICAS → imputación simple y one-hot encoding
cat_cols = df.select_dtypes(include=['object', 'category']).columns
df_cat = df[cat_cols].copy()
df_cat = df_cat.fillna('desconocido')                            # imputar NaN con etiqueta
df_cat = pd.get_dummies(df_cat, drop_first=False, dtype=int)     # codificar categorías

# 3) UNIR → dataset final procesado
df_procesado = pd.concat([df_num, df_cat], axis=1)



### 2.- Realizar ajuste mediante kmeans

Una vez depurado el conjunto de datos, es momento de aplicar el algoritmo de **kmeans**.

1. Ajuste el modelo de **kmeans** sobre el conjunto de datos, con un total de **8 clusters**.
2. Asociar a cada individuo el correspondiente cluster y calcular valor de los centroides de cada cluster.
3. Realizar un resumen de las principales cualidades de cada cluster. Para  esto debe calcular (para cluster) las siguientes medidas de resumen:
    * Valor promedio de las variables numérica
    * Moda para las variables numericas

In [ ]:
# FIXME


# Ajuste KMeans (k = 6) sobre el dataset procesado
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
labels = kmeans.fit_predict(df_procesado)

# Asignar cluster a cada fila (sobre el df original para resumir por tipo de variable)
df_clusters = df.copy()
df_clusters["cluster"] = labels

# Centroides en el espacio procesado
centroides = pd.DataFrame(
    kmeans.cluster_centers_,
    columns=df_procesado.columns,
    index=sorted(np.unique(labels))
)

# Resúmenes por cluster
num_cols = df_clusters.select_dtypes(include=[np.number]).columns.drop("cluster", errors="ignore")
cat_cols = df_clusters.select_dtypes(include=["object", "category"]).columns

resumen_num = df_clusters.groupby("cluster")[num_cols].mean()

def _moda(s: pd.Series):
    m = s.mode(dropna=True)
    return m.iloc[0] if not m.empty else np.nan

resumen_cat = df_clusters.groupby("cluster")[cat_cols].agg(_moda)




### 3.- Elegir Número de cluster

Estime mediante la **regla del codo**, el número de cluster apropiados para el caso.
Para efectos prácticos, eliga la siguiente secuencia como número de clusters a comparar:

$$[5, 10, 20, 30, 50, 75, 100, 200, 300]$$

Una vez realizado el gráfico, saque sus propias conclusiones del caso.

In [ ]:
# FIXME

ks = [5, 10, 20, 30, 50, 75, 100, 200, 300]
inercias = []

for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(df_procesado)
    inercias.append(km.inertia_)

res_elbow = pd.DataFrame({"k": ks, "inercia": inercias})
display(res_elbow)

plt.figure(figsize=(6,4))
plt.plot(ks, inercias, marker="o")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia (SSE)")
plt.title("Regla del codo")
plt.grid(True)
plt.show()


Al observar el gráfico resultante, se pueden obtener conclusiones sobre el número apropiado de clusters. La regla del codo sugiere elegir el número de clusters donde la reducción en la inercia se estabiliza significativamente. En otras palabras, se busca el punto en el gráfico donde la curva de inercia comienza a aplanarse o forma un codo.

## Reducción de Dimensionalidad

<img src="https://1000logos.net/wp-content/uploads/2020/11/Wine-Logo-old.png" width = "300" align="center"/>


Para este ejercicio utilizaremos el **Wine Dataset**, un conjunto de datos clásico disponible en la librería **scikit-learn** y en el repositorio de la **UCI Machine Learning**.
Este dataset contiene información de **178 muestras de vino** provenientes de la región italiana de *Piamonte*. Cada vino pertenece a una de **tres variedades de uva** (*clases*), que actúan como etiquetas para el análisis supervisado, pero aquí se usarán solo como referencia en la visualización.

Cada muestra está descrita por **13 variables químicas** obtenidas de un análisis de laboratorio, entre ellas:

* **Alcohol**: porcentaje de alcohol en el vino.
* **Malic acid**: concentración de ácido málico.
* **Ash**: contenido de ceniza.
* **Alcalinity of ash**: alcalinidad de la ceniza.
* **Magnesium**: cantidad de magnesio (mg/L).
* **Total phenols**: concentración total de fenoles.
* **Flavanoids**: tipo de fenoles con propiedades antioxidantes.
* **Nonflavanoid phenols**: fenoles que no son flavonoides.
* **Proanthocyanins**: compuestos relacionados con el color y el sabor.
* **Color intensity**: intensidad del color del vino.
* **Hue**: matiz del color.
* **OD280/OD315 of diluted wines**: relación de absorbancia que mide la calidad del vino.
* **Proline**: concentración de prolina (un aminoácido).

Estas características permiten representar cada vino como un punto en un espacio de **13 dimensiones**.

El objetivo del análisis con este dataset es **reducir la dimensionalidad** para visualizar y explorar patrones en los datos. Para ello aplicaremos:

* **PCA (Principal Component Analysis):** identificar las combinaciones lineales de variables que explican la mayor varianza en el conjunto.
* **t-SNE (t-distributed Stochastic Neighbor Embedding):** mapear las muestras a 2D o 3D, preservando relaciones de vecindad y estructuras no lineales.

La comparación entre ambas técnicas permitirá observar cómo las tres clases de vinos se diferencian en el espacio reducido y discutir la utilidad de la reducción de dimensionalidad en datos con mayor número de variables que en el caso del dataset *Wine*.



In [ ]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
# cargar dataset
dataset = load_wine()

# nombres de las variables
features = dataset.feature_names
target = 'wine_class'

# construir DataFrame
wine = pd.DataFrame(dataset.data, columns=features)
wine[target] = dataset.target

# ver primeras filas
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,wine_class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0




### 1. **Análisis detallado con PCA**

* Calcular la **varianza explicada** por cada componente principal y representar el gráfico de varianza acumulada, identificando cuántos componentes son necesarios para capturar al menos el **90–95% de la información**.
* Construir tablas y gráficos que muestren cómo las observaciones (vinos) se proyectan en las primeras componentes principales.
* Analizar los **loadings** (coeficientes de cada variable en los componentes) e interpretar qué características químicas del vino (alcohol, fenoles, color, etc.) tienen mayor influencia en las nuevas dimensiones.
* Visualizar los datos reducidos a 2D o 3D e interpretar si las **tres variedades de vino** se separan de forma clara en el espacio proyectado.



In [ ]:
# FIXME
# === PCA: varianza explicada, loadings y visualización 2D/3D ===

# Datos
X = wine[features].values
y = wine[target].values

# Escalado y PCA
Xz = StandardScaler().fit_transform(X)
pca = PCA().fit(Xz)
scores = pca.transform(Xz)

# 1) Varianza explicada y acumulada (+ componentes para ≥90% y ≥95%)
var_exp = pca.explained_variance_ratio_
var_acum = np.cumsum(var_exp)
n90 = np.searchsorted(var_acum, 0.90) + 1
n95 = np.searchsorted(var_acum, 0.95) + 1

tabla_var = pd.DataFrame({
    "PC": np.arange(1, len(var_exp)+1),
    "Var_Exp": var_exp,
    "Var_Acum": var_acum
})
display(tabla_var.head(10))
print(f"Componentes para ≥90%: {n90} | para ≥95%: {n95}")

plt.figure(figsize=(6,4))
plt.bar(np.arange(1, len(var_exp)+1), var_exp)
plt.plot(np.arange(1, len(var_acum)+1), var_acum, marker="o")
plt.axhline(0.90, ls="--")
plt.axhline(0.95, ls="--")
plt.xlabel("Componente principal")
plt.ylabel("Varianza explicada / acumulada")
plt.title("PCA: varianza explicada")
plt.show()

# 2) Loadings (contribuciones de variables a PCs) y ranking por |loading|
loadings = pd.DataFrame(
    pca.components_.T,
    index=features,
    columns=[f"PC{i+1}" for i in range(pca.components_.shape[0])]
)
display(loadings.iloc[:, :5])  # primeras 5 PCs

top_pc1 = loadings['PC1'].abs().sort_values(ascending=False).head(10)
top_pc2 = loadings['PC2'].abs().sort_values(ascending=False).head(10)
print("Top variables por |loading| en PC1:")
display(loadings.loc[top_pc1.index, ['PC1']])
print("Top variables por |loading| en PC2:")
display(loadings.loc[top_pc2.index, ['PC2']])

# 3) Visualización 2D y 3D coloreando por clase
pc_df = pd.DataFrame(scores[:, :3], columns=["PC1","PC2","PC3"])
pc_df["clase"] = y

plt.figure(figsize=(6,4))
for c in np.unique(y):
    m = pc_df["clase"] == c
    plt.scatter(pc_df.loc[m,"PC1"], pc_df.loc[m,"PC2"], s=25, label=str(c), alpha=0.8)
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.title("PCA 2D"); plt.legend(title=target)
plt.show()

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection='3d')
for c in np.unique(y):
    m = pc_df["clase"] == c
    ax.scatter(pc_df.loc[m,"PC1"], pc_df.loc[m,"PC2"], pc_df.loc[m,"PC3"], s=25, label=str(c), alpha=0.8)
ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.set_zlabel("PC3"); ax.set_title("PCA 3D")
ax.legend(title=target)
plt.show()




### 2. **Análisis detallado con t-SNE**

* Aplicar **t-SNE** para reducir los datos a 2 dimensiones, probando diferentes configuraciones de hiperparámetros como *perplexity* y *learning rate*.
* Comparar las distintas visualizaciones obtenidas y discutir cómo los hiperparámetros afectan la estructura de los clústeres.
* Analizar si las **tres clases de vinos** forman agrupaciones definidas y si t-SNE logra capturar relaciones no lineales que PCA no refleja.



In [ ]:
# FIXME
# === t-SNE: proyecciones 2D con distintas configuraciones ===

X = wine[features].values
y = wine[target].values
Xz = StandardScaler().fit_transform(X)

configs = [
    {"perp": 5,  "lr": 200},
    {"perp": 10, "lr": 200},
    {"perp": 30, "lr": 200},
    {"perp": 50, "lr": 200},
    {"perp": 30, "lr": 50},
    {"perp": 30, "lr": 1000},
]

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
axes = axes.ravel()

for ax, cfg in zip(axes, configs):
    tsne = TSNE(
        n_components=2,
        perplexity=cfg["perp"],
        learning_rate=cfg["lr"],
        n_iter=1500,
        init="pca",
        random_state=42,
        verbose=0
    )
    Z = tsne.fit_transform(Xz)
    for c in np.unique(y):
        m = y == c
        ax.scatter(Z[m, 0], Z[m, 1], s=15, alpha=0.85, label=str(c))
    ax.set_title(f"perp={cfg['perp']}, lr={cfg['lr']}")
    ax.set_xticks([]); ax.set_yticks([])

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, title=target, ncol=3, loc="upper center")
fig.suptitle("t-SNE 2D con diferentes hiperparámetros", y=0.98)
plt.tight_layout(rect=[0,0,1,0.95])
plt.show()




### 3. **Comparación entre PCA y t-SNE**

* Contrastar las visualizaciones y discutir las **ventajas y limitaciones** de cada técnica:

  * PCA como método **lineal** para interpretar varianza y relaciones globales.
  * t-SNE como método **no lineal** que preserva relaciones locales y vecindades.
* Evaluar en qué escenarios prácticos sería más recomendable usar PCA (interpretabilidad, reducción previa para modelos) o t-SNE (exploración y visualización de clústeres).
* Reflexionar sobre la **importancia de la reducción de dimensionalidad** en datasets de alta dimensión como Wine, destacando su utilidad para:

  * Visualizar patrones ocultos en los datos.
  * Reducir complejidad y ruido antes de aplicar algoritmos de aprendizaje automático.
  * Facilitar la interpretación y comunicación de resultados.



In [ ]:
# FIXME

#PCA es un método lineal que rota el espacio para maximizar varianza global en ejes ortogonales, entrega varianza explicada (permite decidir cuántas PCs retener) y “loadings” interpretables (qué variables pesan en cada PC); es determinista, rápido y su transformación puede reutilizarse en nuevos datos, por eso es ideal para reducción previa a modelado y para explicar qué variables dominan. t-SNE es un método no lineal orientado a preservar vecindades locales, excelente para visualizar clusters en 2D, pero no conserva distancias/densidades globales, depende de hiperparámetros (perplexity, learning rate) y no ofrece varianza explicada ni una proyección aplicable directamente a datos nuevos; por ello es más una herramienta exploratoria/visual. En Wine, usa PCA para cuantificar información retenida e identificar compuestos relevantes, y t-SNE para revelar separaciones locales entre las tres clases cuando una proyección lineal 2D no las muestra claramente; práctica recomendada: estandarizar, reducir con PCA y, si se busca visualización no lineal, aplicar t-SNE sobre las PCs, siempre reportando hiperparámetros y fijando semilla.